In [1]:

from my_modules.log import log,Limit,CounterBreak,CounterMod



from my_modules.load_data import MyDataset
import my_modules.model.lstm as model_modules
from my_modules.model.lstm import LSTM_0,move_to_gpu,del_gpu,LSTM_1,LSTM_2,LSTM_3,LSTM_4
from my_modules.model.lstmcnn import LSTMCNN_1,LSTMCNN_2,LSTMCNN_3,LSTMCNN_4,LSTMCNN_5
from my_modules.model.lstmcnn_part2 import LSTMCNN_6
from my_modules.model.lstmcnn_part3 import LSTMCNN_10



import torch
from torch import nn
from torch.utils.data import DataLoader

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from tqdm import tqdm

print(model_modules.get_used_memory())

0


In [2]:
train_index_movie = [i for i in range(38)]
test_index_movie = [i for i in range(len(train_index_movie),40)]

data_path = r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore\ori_datasets\s01.dat'
# window_size = 145*5
window_size = 145*10

step_window = 145

data_train = MyDataset(data_path,window_size,step_window,train_index_movie)
data_test = MyDataset(data_path,window_size,step_window,test_index_movie)

In [3]:
model_modules.do_gpu = torch.cuda.is_available()
# model_modules.do_gpu = False

if(model_modules.do_gpu):
    print(torch.cuda.get_device_name())
else:
    print('using CPU')

data_loader = DataLoader(data_train,batch_size = 16,shuffle=True,pin_memory=model_modules.do_gpu)
data_loader_test  = DataLoader(data_test,batch_size = 16,shuffle=True,pin_memory=model_modules.do_gpu)

GeForce GTX 1070 Ti


In [4]:

input_size = 40
torch.manual_seed(3333)
loss_func = nn.MSELoss()

model = LSTMCNN_10(input_size=input_size,number_layers=1,hidden_size=20)
model = move_to_gpu(model)
model.load_state_dict(torch.load(r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore\w\v017\model_070_1dot90'))
error = 0
c=0
temp = []
with torch.no_grad():
    for test_data,y_test in tqdm(data_loader_test):
        test_data = move_to_gpu(test_data)    
        y_test = move_to_gpu(y_test)

        bz = test_data.shape[0]
        model.reset_hidden(bz)
        y_hat_test = model(test_data)
    
        if(CounterMod.init().set_mod(6).count()):
            for i in range(y_hat_test.shape[0]):
                
                print(y_hat_test[i],'vs',y_test[i])
        batch_loss_test = loss_func(y_hat_test,y_test)
        for eee in (y_hat_test[:,0]-y_hat_test[:,1]).tolist():
            temp.append(eee)
        
        error+= float(batch_loss_test)
        c+=1



print(error/c)
print('max diff',max(temp) )
print('mean diff',sum(temp)/len(temp) )

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.00it/s]

tensor([7.1618, 6.4838], device='cuda:0') vs tensor([5.1000, 7.1200], device='cuda:0')
tensor([3.9602, 4.6790], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([6.1016, 5.7010], device='cuda:0') vs tensor([5.1000, 7.1200], device='cuda:0')
tensor([4.5375, 4.7681], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.7943, 5.5273], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.9268, 5.5600], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.0020, 4.5409], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.9095, 5.8547], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.9155, 5.6466], device='cuda:0') vs tensor([5.1000, 7.1200], device='cuda:0')
tensor([4.3096, 5.3126], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([4.4378, 5.4611], device='cuda:0') vs tensor([3.2400, 6.1800], device='cuda:0')
tensor([3.9229, 4.9547], device='cuda:0') v